In [19]:
# Part 1: Import Libraries and Define the Cage Class

import copy
import random
import operator
from functools import reduce

class Cage:
    def __init__(self, cells, operation, target):
        self.cells = cells
        self.operation = operation
        self.target = target


In [20]:
# Part 2: Function to Generate KenKen Puzzle

def generate_KenKen(size):
    grid = [[0] * size for _ in range(size)]
    fill_grid_with_numbers(grid, size)
    cages = generate_random_cages(grid, size)
    return [[0] * size for _ in range(size)], cages  # Return unsolved grid

# Part 3: Function to Fill Grid with Numbers

def fill_grid_with_numbers(grid, size):
    """
    This function fills the grid with numbers 1 to size such that
    each row and each column has unique values. It uses backtracking
    to ensure all cells are filled with valid numbers.
    """

    def is_safe_to_place(grid, row, col, num):
        # Check if 'num' is not present in the current row
        if num in grid[row]:
            return False
        # Check if 'num' is not present in the current column
        for r in range(size):
            if grid[r][col] == num:
                return False
        return True

    def fill_grid_backtracking(grid, row, col):
        if row == size:  # If all rows are filled, the grid is complete
            return True
        if col == size:  # Move to the next row
            return fill_grid_backtracking(grid, row + 1, 0)

        # Try placing each number from 1 to size in the current cell
        numbers = list(range(1, size + 1))
        random.shuffle(numbers)  # Shuffle to introduce randomness

        for num in numbers:
            if is_safe_to_place(grid, row, col, num):
                grid[row][col] = num
                if fill_grid_backtracking(grid, row, col + 1):
                    return True
                # Undo assignment (backtrack)
                grid[row][col] = 0

        return False

    # Start backtracking from the first cell
    fill_grid_backtracking(grid, 0, 0)

    # Part 4: Function to Generate Random Cages

def generate_random_cages(grid, size):
    cages = []
    visited = [[False] * size for _ in range(size)]

    for i in range(size):
        for j in range(size):
            if not visited[i][j]:
                cage_size = random.randint(1, size)
                cells = [(i, j)]
                visited[i][j] = True

                while len(cells) < cage_size:
                    x, y = cells[-1]
                    neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]
                                 if 0 <= x + dx < size and 0 <= y + dy < size and not visited[x + dx][y + dy]]
                    if neighbors:
                        next_cell = random.choice(neighbors)
                        cells.append(next_cell)
                        visited[next_cell[0]][next_cell[1]] = True
                    else:
                        break

                operation = random.choice(['+', '*']) if cage_size > 2 else random.choice(['+', '-', '*', '/'])
                target = calculate_target(grid, cells, operation)
                cages.append(Cage(cells, operation, target))

    return cages


# Part 5: Function to Calculate Cage Target

def calculate_target(grid, cells, operation):
    values = [grid[x][y] for x, y in cells]
    if operation == '+':
        return sum(values)
    elif operation == '-':
        if len(values) == 1:
            return values[0]
        return abs(values[0] - values[1])
    elif operation == '*':
        return reduce(operator.mul, values, 1)
    elif operation == '/':
        if len(values) == 1:
            return values[0]
        return max(values) // min(values)
    return None




In [21]:
# Part 6: KenKen Backtracking Solver

def solve_kenken(grid, cages):
    row, col = find_unassigned_location(grid)
    if row == -1 and col == -1:
        return True

    for num in range(1, len(grid) + 1):
        if is_safe_kenken(grid, row, col, num, cages):
            grid[row][col] = num

            if solve_kenken(grid, cages):
                return True

            grid[row][col] = 0

    return False

# Part 7: Utility Functions for KenKen Solver

def is_safe_kenken(grid, row, col, num, cages):
    # Row and column uniqueness check
    if num in grid[row] or any(grid[i][col] == num for i in range(len(grid))):
        return False

    # Validate within the cage
    for cage in cages:
        if (row, col) in cage.cells:
            cage_values = [grid[x][y] for x, y in cage.cells if grid[x][y] != 0] + [num]
            if len(cage_values) == len(cage.cells):
                if not validate_cage_operation(cage.operation, cage_values, cage.target):
                    return False
    return True

def validate_cage_operation(operation, values, target):
    if len(values) == 1:
        return target == values[0]
    if operation == '+':
        return sum(values) == target
    elif operation == '-':
        return abs(values[0] - values[1]) == target
    elif operation == '*':
        return reduce(operator.mul, values, 1) == target
    elif operation == '/':
        larger, smaller = max(values), min(values)
        return larger // smaller == target
    return False

def find_unassigned_location(grid):
    for i, row in enumerate(grid):
        if 0 in row:
            return i, row.index(0)
    return -1, -1


In [22]:
# Part 8: KenKen Domain Constraint Solver

def solve_kenken_csp(grid, cages):
    """
    Function to solve the KenKen grid using Constraint Satisfaction Problem (CSP)
    """

    def create_domains(grid):
        """
        Function to create domains for each cell in the grid
        """
        domains = {}
        for i in range(len(grid)):
            for j in range(len(grid)):
                if grid[i][j] == 0:
                    available_vals = []
                    for k in range(1, len(grid) + 1):
                        if is_safe_kenken(grid, i, j, k, cages):
                            available_vals.append(k)
                    domains[(i, j)] = available_vals
                else:
                    domains[(i, j)] = grid[i][j]
        return domains

    def is_valid_assignment(i, j, val, assignment):
        """
        Function to check if assigning a value to a cell is valid
        """
        # Row and column uniqueness check
        for k in range(len(grid)):
            if assignment[(i, k)] == val or assignment[(k, j)] == val:
                return False

        # Validate within the cage
        for cage in cages:
            if (i, j) in cage.cells:
                cage_values = [assignment[(x, y)] for x, y in cage.cells if (x, y) in assignment and type(assignment[(x, y)]) != list]
                cage_values.append(val)
                if len(cage_values) == len(cage.cells):
                    if not validate_cage_operation(cage.operation, cage_values, cage.target):
                        return False

        return True

    def find_unassigned_location(assignment):
        """
        Function to find an unassigned location in the grid
        """
        for i in range(len(grid)):
            for j in range(len(grid)):
                if type(assignment[(i, j)]) == list:
                    return (i, j)
        return (-1, -1)

    # Recursive function to solve the KenKen grid using CSP
    def solve_csp(assignment):
        i, j = find_unassigned_location(assignment)
        if i == -1 and j == -1:
            return True  # If no unassigned location is found, the grid is solved

        # Try assigning each possible value to the unassigned location
        possible_vals = assignment[(i, j)].copy()
        for val in possible_vals:
            if is_valid_assignment(i, j, val, assignment):
                assignment[(i, j)] = val
                global steps
                steps += 1

                if solve_csp(assignment):
                    return True
                assignment[(i, j)] = possible_vals  # Backtrack
                steps += 1

        return False

    # Create initial domains for each cell in the grid
    domains = create_domains(grid)
    assignment = {(i, j): val for (i, j), val in domains.items()}
    global steps
    steps = 0

    # Solve the KenKen grid using CSP
    if solve_csp(assignment):
        solved_grid = [[assignment[(i, j)] for j in range(len(grid))] for i in range(len(grid))]
        return (solved_grid, steps)
    else:
        return None


In [23]:
# Part 9: Print Solution

def print_solution(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))


In [24]:
# Part 10: Run Example

size = 5
unsolved_grid, kenken_cages = generate_KenKen(size)

print("Generated KenKen Cages:")
for cage in kenken_cages:
    print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target}")
print("-----------------------")

# Solve using Backtracking solver
backtrack_grid = copy.deepcopy(unsolved_grid)
if solve_kenken(backtrack_grid, kenken_cages):
    print("Solved KenKen Puzzle (BackTracking):")
    print_solution(backtrack_grid)
else:
    print("No solution found using BackTracking.")
print("-----------------------")

# Solve using CSP solver
csp_solved, steps = solve_kenken_csp(unsolved_grid, kenken_cages)
if csp_solved:
    print("Solved KenKen Puzzle (CSP):")
    print_solution(csp_solved)
else:
    print("No solution found using CSP.")


Generated KenKen Cages:
Cage Cells: [(0, 0), (1, 0), (2, 0), (3, 0)], Operation: *, Target: 30
Cage Cells: [(0, 1), (0, 2), (0, 3), (1, 3), (1, 4)], Operation: +, Target: 17
Cage Cells: [(0, 4)], Operation: +, Target: 5
Cage Cells: [(1, 1), (2, 1)], Operation: /, Target: 1
Cage Cells: [(1, 2), (2, 2)], Operation: /, Target: 1
Cage Cells: [(2, 3), (2, 4)], Operation: -, Target: 3
Cage Cells: [(3, 1), (3, 2)], Operation: *, Target: 20
Cage Cells: [(3, 3), (4, 3), (4, 4)], Operation: *, Target: 6
Cage Cells: [(3, 4)], Operation: *, Target: 2
Cage Cells: [(4, 0), (4, 1), (4, 2)], Operation: *, Target: 20
-----------------------
Solved KenKen Puzzle (BackTracking):
2 1 4 3 5
1 2 3 5 4
5 3 2 4 1
3 4 5 1 2
4 5 1 2 3
-----------------------
Solved KenKen Puzzle (CSP):
2 1 4 3 5
1 2 3 5 4
5 3 2 4 1
3 4 5 1 2
4 5 1 2 3
